在前面的例子中, 我们从依赖项 ("可依赖对象") 中返回了一个 dict:

但是后面我们在路径操作函数的参数 commons 中得到了一个 dict。

我们知道编辑器不能为 dict 提供很多支持(比如补全)，因为编辑器不知道 dict 的键和值类型。

对此，我们可以做的更好...

到目前为止，您看到的依赖项都被声明为函数。

但这并不是声明依赖项的唯一方法(尽管它可能是更常见的方法)。

关键因素是依赖项应该是 "可调用

Python 中的 "可调用对象" 是指任何 Python 可以像函数一样 "调用" 的对象。

所以，如果你有一个对象 something (可能不是一个函数)，你可以 "调用" 它(执行它)，就

something()

或者

something(some_argument, some_keyword_argument="foo")

这就是 "可调用对象"。像：对象"。

您可能会注意到，要创建一个 Python 类的实例，您可以使用相同的语法。

举个例子:

In [ ]:
class Cat:
    def __init__(self, name: str):
        self.name = name


fluffy = Cat(name="Mr Fluffy")

在这个例子中, fluffy 是一个 Cat 类的实例。

为了创建 fluffy，你调用了 Cat 。

所以，Python 类也是 可调用对象。

因此，在 FastAPI 中，你可以使用一个 Python 类作为一个依赖项。

实际上 FastAPI 检查的是它是一个 "可调用对象"（函数，类或其他任何类型）以及定义的参数。

如果您在 FastAPI 中传递一个 "可调用对象" 作为依赖项，它将分析该 "可调用对象" 的参数，并以处理路径操作函数的参数的方式来处理它们。包括子依赖项。

这也适用于完全没有参数的可调用对象。这与不带参数的路径操作函数一样。

所以，我们可以将上面的依赖项 "可依赖对象" common_parameters 更改为类 Comm

注意用于创建类实例的 __init__ 方法：

它与我们以前的 common_parameters 具有相同的参数：

这些参数就是 FastAPI 用来 "处理" 依赖项的。

在两个例子下，都有：

一个可选的 q 查询参数，是 str 类型。
一个 skip 查询参数，是 int 类型，默认值为 0。
一个 limit 查询参数，是 int 类型，默认值为 100。
在两个例子下，数据都将被转换、验证、在 OpenAPI schema 上文档化，等等。onQueryParams:

In [1]:
import uvicorn
from fastapi import Depends, FastAPI
app = FastAPI()

class CommonQueryParams:
    def __init__(self, q: str | None = None, skip: int = 0, limit: int = 100):
        self.q = q
        self.skip = skip
        self.limit = limit


@app.get("/items/")
async def read_items(commons: CommonQueryParams = Depends(CommonQueryParams)):
    return commons


@app.get("/users/")
async def read_users(commons: CommonQueryParams = Depends(CommonQueryParams)):
    return commons

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [22084]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:65386 - "GET /users/?q=%E6%9D%8E%E5%9B%9B HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [22084]


FastAPI 调用 CommonQueryParams 类。

这将创建该类的一个 "实例"，该实例将作为参数 commons 被传递给你的函数。

In [ ]:
url = 'http://127.0.0.1:8009/items/?q=张三' 
res = requests.get(url) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"q":"张三","skip":0,"limit":100}'

In [ ]:
url = 'http://127.0.0.1:8009/users/?q=李四' 
res = requests.get(url) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"q":"李四","skip":0,"limit":100}'

## 类型注解 vs Depends

注意，我们在上面的代码中编写了两次CommonQueryParams：

commons: CommonQueryParams = Depends(CommonQueryParams)

你可以简写成：
commons = Depends(CommonQueryParams)
或者
commons: CommonQueryParams = Depends()

但是更推荐写成 commons: CommonQueryParams = Depends()
因为，声明类型是被鼓励的，因为那样你的编辑器就会知道将传递什么作为参数 commons ，然后它可以帮助你完成代码，类型检查，等等。